📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn import datasets

In [2]:
# set a seed for deterministic results
seed = 42

# Logistic Regression

<div style="display:flex; margin-top:50px;">
   <div style="width:20%; margin-right:auto; margin-left:auto;">
      <table style="margin:0 auto; width:80%; text-align:center">
         <caption style="font-weight:bold;">Dataset</caption>
         <thead>
            <tr>
               <th style="width:25%; text-align:center"><span style="color:magenta;">#</span></th>
               <th style="width:25%; text-align:center"><span style="color:#9090ff;">x<sub>1</sub></span></th>
               <th style="width:25%; text-align:center"><span style="color:#9090ff;">x<sub>2</sub></span></th>
               <th style="width:25%; text-align:center"><span style="color:red;">y</span></th>
            </tr>
         </thead>
         <tbody>
            <tr><th>1</th><td>1</td><td>1</td><td>0</td></tr>
            <tr><th>2</th><td>2</td><td>3</td><td>1</td></tr>
            <tr><th>3</th><td>1</td><td>2</td><td>0</td></tr>
            <tr><th>4</th><td>3</td><td>1</td><td>0</td></tr>
            <tr><th>5</th><td>2</td><td>4</td><td>1</td></tr>
            <tr><th>6</th><td>3</td><td>2</td><td>1</td></tr>
            <tr><th>7</th><td>4</td><td>1</td><td>1</td></tr>
         </tbody>
      </table>
   </div>
   <div style="width:80%; padding:10px;">
      <figure style="text-align:center; margin:0;">
         <img src="../assets/images/original/perceptron/logistic-regression.svg" alt="logistic-regression.svg" style="max-width:80%; height:auto;">
         <figcaption style="font-size:smaller; text-align:center;">Logistic Regression Model</figcaption>
      </figure>
   </div>
</div>

In [ ]:
# generate artificial data
n_samples, n_features = 10, 2

x, y = datasets.make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=seed,
)

# convert numpy.ndarray to torch.Tensor
x_train = torch.from_numpy(x.astype(np.float32))
y_train = torch.from_numpy(y.astype(np.float32)).view(-1, 1)

# plot
plt.scatter(x[y == 0][:, 0], x[y == 0][:, 1], color="b", label="Class 0")
plt.scatter(x[y == 1][:, 0], x[y == 1][:, 1], color="r", label="Class 1")
plt.xlabel("x_1")
plt.ylabel("x_2")
plt.legend()
plt.show()

In [ ]:
# logistic regression model
model = torch.nn.Sequential(torch.nn.Linear(n_features, 1), torch.nn.Sigmoid())

model

In [5]:
# plot stuff
state = []

In [ ]:
# initial weights [educational purpose]
with torch.no_grad():
    # b [w_0] = +1
    model[0].bias[0].fill_(1)

    # w_1 = -1
    model[0].weight[0, 0].fill_(-1)

    # w_2 = +1
    model[0].weight[0, 1].fill_(1)

# hyper parameters
epoch = 6
lr = 0.5
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# training loop
model.train()
for i in range(epoch):

    # forward
    y_pred = model(x_train)

    # backward
    loss = criterion(y_pred, y_train)
    loss.backward()

    # save new y_pred every 5 epochs [plot stuff]
    state.append([model[0].weight.clone().detach().numpy(), model[0].bias.clone().detach().numpy()])

    # update parameters
    optimizer.step()
    optimizer.zero_grad()

    # log
    print(f"epoch {i+1:0{len(str(epoch))}:>2}/{epoch}  ->  loss: {loss.item():>7.5f}")

In [ ]:
# plot
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 16), layout="compressed")

for row in range(3):
    for col in range(2):
        axs[row, col].scatter(x[y == 0][:, 0], x[y == 0][:, 1], color="b", label="Class 0")
        axs[row, col].scatter(x[y == 1][:, 0], x[y == 1][:, 1], color="r", label="Class 1")
        axs[row, col].set(
            title=f"epoch {row * 2 + col}, W: {state[row * 2 + col][0].squeeze()}, b: {state[row * 2 + col][1].squeeze():.3f}",
            xlim=(x[:, 0].min() - 1, x[:, 0].max() + 1),
            ylim=(x[:, 1].min() - 1, x[:, 1].max() + 1),
        )

        # decision boundary
        w, b = state[row * 2 + col]
        slope = -w[0][0] / w[0][1]
        intercept = -b[0] / w[0][1]
        x_plot = np.array([np.min(x[:, 0]), np.max(x[:, 0])])
        y_plot = slope * x_plot + intercept

        axs[row, col].plot(x_plot, y_plot, color="g", linestyle="--", label="Decision Boundary")
        axs[row, col].legend()

plt.show()